In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os
from efficientnet_pytorch import EfficientNet

# Define the dataset class for training data
class EmbryoDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Map day names to indices
        day_index = {'day_1': 0, 'day_2': 1, 'day_3': 2, 'day_4': 3}
        
        # Iterate through each day folder
        for day in os.listdir(root_dir):
            day_path = os.path.join(root_dir, day)
            if os.path.isdir(day_path):
                label = day_index[day]  # Map day name to index
                
                # Iterate through images in the day folder
                for image_name in os.listdir(day_path):
                    image_path = os.path.join(day_path, image_name)
                    self.images.append(image_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformation for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define paths
train_path = "/kaggle/input/abdous-egg-experiment/dataset/dataset"
test_path = "/kaggle/input/abdous-egg-experiment/dataset/test"

# Create dataset and data loader for training data
train_dataset = EmbryoDataset(root_dir=train_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create test dataset and data loader
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_name = self.images[idx]
        image_path = os.path.join(self.root_dir, image_name)
        image = Image.open(image_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, image_name

test_dataset = TestDataset(root_dir=test_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained EfficientNet model
efficientnet = EfficientNet.from_pretrained('efficientnet-b0', num_classes=4)  # 4 classes: day_1, day_2, day_3, day_4

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientnet.parameters(), lr=0.001)

# Train the model for 7 epochs
num_epochs = 7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet.to(device)
for epoch in range(num_epochs):
    efficientnet.train()  # Set model to training mode
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = efficientnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Print loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions for each image in the test set
predictions = []
efficientnet.eval()  # Set model to evaluation mode
for images, image_names in test_loader:
    with torch.no_grad():
        images = images.to(device)
        outputs = efficientnet(images)
        _, predicted = torch.max(outputs, 1)
        predicted_day = predicted + 1  # Convert class index to day (1-indexed)
        for image_name, day in zip(image_names, predicted_day):
            predictions.append((image_name, day.item()))

# Generate submission file
submission_df = pd.DataFrame(predictions, columns=['ID', 'target'])
submission_df.to_csv('submission_efficientnet.csv', index=False)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:01<00:00, 17.1MB/s]


Loaded pretrained weights for efficientnet-b0
Epoch [1/7], Loss: 0.2750
Epoch [2/7], Loss: 0.1849
Epoch [3/7], Loss: 0.0363
Epoch [4/7], Loss: 0.0589
Epoch [5/7], Loss: 0.0072
Epoch [6/7], Loss: 0.1066
Epoch [7/7], Loss: 0.0914


In [2]:
pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=ca105f5cc24d5783a43ca308d66ad632ec26a5fc030e9f7cfb5af05fe573477e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.
